In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [32]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [33]:
class LeNet5(nn.Module):

    def __init__(self, num_classes):
        super(LeNet, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=num_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)

        return x

In [34]:
LEARNING_RATE = 0.001
BATCH_SIZE = 32
NUM_EPOCHS = 15
IMAGE_SIZE = 32
NUM_CLASSES = 10

In [35]:
x = torch.randn(64, 1, IMAGE_SIZE, IMAGE_SIZE)
model = LeNet(num_classes=NUM_CLASSES)
print(model(x).shape)

torch.Size([64, 10])


In [36]:
transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), 
    transforms.ToTensor()
    ])

train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms, download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [37]:
model = LeNet5(NUM_CLASSES).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [30]:
model.train()
running_loss = 0
train_losses = []

for epoch in range(NUM_EPOCHS):
    for data, targets in train_loader:
        data = data.to(DEVICE)
        targets = targets.to(DEVICE)

        scores = model(data)
        loss = criterion(scores, targets)
        running_loss += loss.item() * data.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)

KeyboardInterrupt: 

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

In [12]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data


NameError: name 'device' is not defined

In [22]:
num_correct = 0
num_samples = 0

model.eval()
with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(DEVICE)
        targets = targets.to(DEVICE)

        socres = model(x)
        _, predictions = scores.max(1)
        num_correct += (predictions == targets).sum()
        num_samples += predictions.size(0)
    
    print(f'{num_correct} / {num_samples}, accuracy: {num_correct/(num_samples)*100.0:.2f}')

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _thnn_conv2d_forward